<a href="https://colab.research.google.com/github/lmudu2/tomato/blob/master/tomato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi
!pip install tensorflow-gpu

Wed Sep 16 19:44:10 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.3.0'

In [ ]:
from tensorflow.keras.layers import Dense,Input,Lambda,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
IMAGE_SIZE = [224,224]

train_path ='/content/drive/My Drive/259770_544347_bundle_archive/New Plant Diseases Dataset(Augmented)/train'
valid_path = '/content/drive/My Drive/259770_544347_bundle_archive/New Plant Diseases Dataset(Augmented)/valid'

In [ ]:
inception = InceptionV3(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

In [ ]:
for layer in inception.layers:
  layer.trainable = False

In [ ]:
folders =glob('/content/drive/My Drive/259770_544347_bundle_archive/New Plant Diseases Dataset(Augmented)/train/*')

In [ ]:
x =Flatten()(inception.output)

In [ ]:
pred = Dense(len(folders),activation='softmax')(x)

In [ ]:
model = Model(inputs=inception.input,outputs=pred)

In [ ]:
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 111, 111, 32) 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 111, 111, 32) 0           batch_normalization_188[0][0]    
_______________________________________________________________________________________

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
trian_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

test_datagen =ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = trian_datagen.flow_from_directory('/content/drive/My Drive/259770_544347_bundle_archive/New Plant Diseases Dataset(Augmented)/train',
                                                 target_size=(224,224),batch_size= 16, class_mode ='categorical')

Found 8295 images belonging to 10 classes.


In [ ]:
valid_set = test_datagen.flow_from_directory('/content/drive/My Drive/259770_544347_bundle_archive/New Plant Diseases Dataset(Augmented)/valid',
                                             target_size = (224,224),
                                             batch_size =16,
                                             class_mode = 'categorical')

Found 4585 images belonging to 10 classes.


In [ ]:
r =model.fit_generator(training_set,validation_data=valid_set,
                       epochs=5,steps_per_epoch=len(training_set),
                       validation_steps=len(valid_set))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
519/519 [==============================] - 8021s 15s/step - loss: 1.8794 - accuracy: 0.7942 - val_loss: 96.0594 - val_accuracy: 0.3939
Epoch 2/5
519/519 [==============================] - 121s 233ms/step - loss: 1.6466 - accuracy: 0.8661 - val_loss: 96.9611 - val_accuracy: 0.4270
Epoch 3/5
519/519 [==============================] - 121s 233ms/step - loss: 1.2973 - accuracy: 0.9014 - val_loss: 103.0144 - val_accuracy: 0.4020
Epoch 4/5
519/519 [==============================] - 121s 233ms/step - loss: 1.3481 - accuracy: 0.9069 - val_loss: 102.6885 - val_accuracy: 0.4382
Epoch 5/5
519/519 [==============================] - 122s 236ms/step - loss: 1.1934 - accuracy: 0.9201 - val_loss: 101.7782 - val_accuracy: 0.4517
